In [0]:
spark.range(5).display()

In [0]:
dbutils.fs.ls("/Volumes/workspace/main/project_data")

In [0]:
transactions_raw = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("/Volumes/workspace/main/project_data/transactions.csv")

display(transactions_raw)


In [0]:
transactions_raw.printSchema()
transactions_raw.count()

## Normalize Column Names

In [0]:
from pyspark.sql.functions import col, to_timestamp

transactions = transactions_raw \
    .withColumnRenamed("cc_num", "customer_id") \
    .withColumnRenamed("trans_num", "transaction_id") \
    .withColumnRenamed("merchant", "merchant_id") \
    .withColumnRenamed("amt", "amount") \
    .withColumnRenamed("is_fraud", "label") \
    .withColumn("event_time", to_timestamp("trans_date")) \
    .drop("first", "last", "trans_time")


In [0]:
transactions.printSchema()
display(transactions)

## Write to Delta Lake

In [0]:
transactions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("transactions")


In [0]:
transactions = spark.read.table("transactions")


In [0]:
spark.sql("SELECT COUNT(*) FROM transactions").show()
spark.sql("SELECT * FROM transactions LIMIT 5").display()


## Feature Engineering (Customer-Level)

In [0]:
from pyspark.sql.functions import *

transactions = spark.table("transactions")

customer_features = transactions.groupBy("customer_id").agg(
    count("*").alias("tx_count"),
    avg("amount").alias("avg_amount"),
    sum("amount").alias("total_spend"),
    stddev("amount").alias("amount_std"),
    countDistinct("merchant_id").alias("unique_merchants")
)

display(customer_features)


In [0]:
customer_features.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("customer_features")


## Transaction risk scoring

In [0]:
scored_transactions = transactions.join(
    customer_features,
    "customer_id",
    "left"
).withColumn(
    "risk_score",
    when(col("amount") > col("avg_amount") + 3*col("amount_std"), 90)
    .when(col("amount") > col("avg_amount") + 2*col("amount_std"), 70)
    .when(col("amount") > col("avg_amount") + col("amount_std"), 40)
    .otherwise(10)
)


In [0]:
display(
    scored_transactions.select(
        "customer_id", "merchant_id", "amount",
        "avg_amount", "risk_score", "label"
    )
)


In [0]:
scored_transactions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("scored_transactions")


## Merchant Anomaly Profiling

In [0]:
merchant_profile = scored_transactions.groupBy("merchant_id").agg(
    avg("risk_score").alias("avg_risk_score"),
    count("*").alias("tx_volume"),
    sum("amount").alias("total_revenue"),
    sum("label").alias("fraud_count")
)

display(merchant_profile.orderBy(desc("avg_risk_score")))


In [0]:
merchant_profile.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("merchant_profile")
